## Load useful packages

In [1]:
# load useful packages
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
%matplotlib inline

## CMIP6 online catalogue

In [8]:
# open cmip6 online catalogue
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [10]:
cat = col.search(source_id=['CNRM-ESM2-1'], 
                 experiment_id=['historical'], table_id=['Ofx'], 
                 variable_id=['areacello']) 
                 #member_id=['r1i1p1f1'])
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r1i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181206
1,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r5i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
2,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r3i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
3,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r2i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
4,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r4i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
5,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r10i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
6,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r9i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
7,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r8i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
8,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r6i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
9,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r11i1p1f2,Ofx,areacello,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200408


In [11]:
# Create dictionary from the list of datasets we found
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [13]:
areacello = dset_dict[list(dset_dict.keys())[0]]

In [14]:
# choose one member_id
areacello = areacello.isel(member_id = 0).squeeze()
areacello

<xarray.Dataset>
Dimensions:     (y: 294, x: 362, nvertex: 4)
Coordinates:
    bounds_lat  (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon  (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat         (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
    lon         (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
    member_id   <U9 'r1i1p1f2'
Dimensions without coordinates: y, x, nvertex
Data variables:
    areacello   (y, x) float32 dask.array<chunksize=(294, 362), meta=np.ndarray>
Attributes: (12/47)
    parent_activity_id:      CMIP
    parent_source_id:        CNRM-ESM2-1
    forcing_index:           2
    product:                 model-output
    experiment:              all-forcing simulation of the recent past
    physics_index:           1
    ...                      ...
    parent_time_units:       days since 1850-01-01 00:00:00
    references:              http://www.umr-cnrm.fr/cmip6/references
    table_id:                Ofx
    activity_id:             CMIP
    realm:                   ocean
    intake_esm_dataset_key:  CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.Ofx.gn

## Select only the Greenland Sea area

In [15]:
GLarea = areacello.areacello.where((areacello.lat>=70) & (areacello.lat<=85) & (areacello.lon <= 10) & (areacello.lon >= -10), drop = True)

## Import NO3 concentration data

In [17]:
cat2 = col.search(source_id=['CNRM-ESM2-1'], 
                 experiment_id=['historical'], table_id=['Omon'], 
                 variable_id=['no3']) 
                 
cat2.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r1i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181206
1,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r3i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
2,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r2i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
3,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r5i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
4,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r4i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190125
5,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r10i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
6,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r9i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
7,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r8i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
8,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r7i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117
9,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r6i1p1f2,Omon,no3,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20200117


In [18]:
# Create dictionary from the list of datasets we found
dsi_dict = cat2.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [26]:
no3 = dsi_dict[list(dsi_dict.keys())[0]]

In [27]:
no3

<xarray.Dataset>
Dimensions:      (y: 294, x: 362, nvertex: 4, lev: 75, axis_nbounds: 2,
                  member_id: 11, time: 1980)
Coordinates:
    bounds_lat   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
  * lev          (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    lev_bounds   (lev, axis_nbounds) float64 dask.array<chunksize=(75, 2), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
  * time         (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds  (time, axis_nbounds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id    (member_id) <U9 'r1i1p1f2' 'r3i1p1f2' ... 'r11i1p1f2'
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    no3          (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 4, 75, 294, 362), meta=np.ndarray>
Attributes: (12/56)
    creation_date:           2020-01-31T12:22:05Z
    parent_activity_id:      CMIP
    parent_source_id:        CNRM-ESM2-1
    realization_index:       11
    forcing_index:           2
    status:                  2021-11-05;created; by gcs.cmip6.ldeo@gmail.com
    ...                      ...
    references:              http://www.umr-cnrm.fr/cmip6/references
    table_id:                Omon
    activity_id:             CMIP
    realm:                   ocnBgchem
    branch_time_in_parent:   47481.0
    intake_esm_dataset_key:  CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.Omon.gn

## Average over ensemble means

In [36]:
no3 = no3.isel(lev = 0)

ValueError: Dimensions {'lev'} do not exist. Expected one or more of Frozen({'y': 294, 'x': 362, 'nvertex': 4, 'axis_nbounds': 2, 'member_id': 11, 'time': 1980})

In [37]:
no3

<xarray.Dataset>
Dimensions:      (y: 294, x: 362, nvertex: 4, axis_nbounds: 2, member_id: 11,
                  time: 1980)
Coordinates:
    bounds_lat   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
    lev          float64 0.5058
    lev_bounds   (axis_nbounds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
  * time         (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds  (time, axis_nbounds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id    (member_id) <U9 'r1i1p1f2' 'r3i1p1f2' ... 'r11i1p1f2'
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    no3          (member_id, time, y, x) float32 dask.array<chunksize=(1, 4, 294, 362), meta=np.ndarray>
Attributes: (12/56)
    creation_date:           2020-01-31T12:22:05Z
    parent_activity_id:      CMIP
    parent_source_id:        CNRM-ESM2-1
    realization_index:       11
    forcing_index:           2
    status:                  2021-11-05;created; by gcs.cmip6.ldeo@gmail.com
    ...                      ...
    references:              http://www.umr-cnrm.fr/cmip6/references
    table_id:                Omon
    activity_id:             CMIP
    realm:                   ocnBgchem
    branch_time_in_parent:   47481.0
    intake_esm_dataset_key:  CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.Omon.gn

In [38]:
# average over the ensemble means
no3_mean = no3.mean(dim = ['member_id'], keep_attrs=True)
no3_mean

<xarray.Dataset>
Dimensions:      (y: 294, x: 362, nvertex: 4, axis_nbounds: 2, time: 1980)
Coordinates:
    bounds_lat   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon   (y, x, nvertex) float64 dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
    lev          float64 0.5058
    lev_bounds   (axis_nbounds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
  * time         (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds  (time, axis_nbounds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    no3          (time, y, x) float32 dask.array<chunksize=(4, 294, 362), meta=np.ndarray>
Attributes: (12/56)
    creation_date:           2020-01-31T12:22:05Z
    parent_activity_id:      CMIP
    parent_source_id:        CNRM-ESM2-1
    realization_index:       11
    forcing_index:           2
    status:                  2021-11-05;created; by gcs.cmip6.ldeo@gmail.com
    ...                      ...
    references:              http://www.umr-cnrm.fr/cmip6/references
    table_id:                Omon
    activity_id:             CMIP
    realm:                   ocnBgchem
    branch_time_in_parent:   47481.0
    intake_esm_dataset_key:  CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.Omon.gn